# Анализ выборов на предмет фальсификации

## 1. Скачка данных с [сайта ЦИК](http://www.vybory.izbirkom.ru)

Изучаем структуру сайта. В ходе её изучения понимаем, что у каждого субъекта есть несколько больших изберательных комиссий, каждая из которых включает в себя какое-то количество УИКов.

Значит парсер должен иметь следущую структуру: 
1. Выбираем конкретные выборы
2. Переходим к конкретному субъекту
3. Переходим к конкретному участку
4. Собираем данные по УИКАМ

In [2]:
import requests
import numpy as np
import datetime
import re
from bs4 import BeautifulSoup

import time
import pandas as pd

In [3]:
start_page_1 = "http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&root=1&tvd=100100031793509&vrn=100100031793505&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=100100031793509&type=227"

На стартовой страничке нас интересуют все ссылки на субъекты. Найдём их и заберём себе. Напишем функцию, которая собирает все субъекты. 

In [5]:
# Забираем себе страничку тремя стандартными запросами: 
response = requests.get(start_page_1)
html = response.content
soup = BeautifulSoup(html,"lxml")
soup

<!DOCTYPE HTML>
<html>
<head>
<base href="http://www.vybory.izbirkom.ru/"/>
<link href="/css/report.css" rel="stylesheet" type="text/css"/>
<title>Сведения о проводящихся выборах и референдумах</title>
<meta content="no-cache" http-equiv="pragma"/>
<meta content="no-cache" http-equiv="cache-control"/>
<meta content="0" http-equiv="expires"/>
<script type="text/javascript">

	var st1 = null; // переменная, хранящая ссылку на SortableTable, !=null для сортируемых таблиц

	function getXls() {

		var sortorder = "0";
		if ((st1!=null)&&(st1.sortDesc!=null)) {

			if (st1.sortDesc) {
				sortorder = "1";
			}
			else if (!st1.sortDesc) {
				sortorder = "-1";
			}
		}

		

		window.location.assign("http://www.vybory.izbirkom.ru/servlet/ExcelReportVersion?"+
								"region=0&"+
								"sub_region=0&"+
								"root=1&"+
						
								"global=1&"+
								"vrn=100100031793505&"+
								"tvd=100100031793509&"+
								"type=227&"+
								"vibid=100100031793509&"+
								"condition=&"+
		

```<a style="text-decoration: none" href="http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&amp;tvd=100100031793509&amp;vrn=100100031793505&amp;region=0&amp;global=1&amp;sub_region=0&amp;prver=0&amp;pronetvd=null&amp;vibid=100100031793851&amp;type=227">Республика Адыгея (Адыгея)</a>```

In [6]:
pre_subjhrefs = soup.findAll("a",{'style':'text-decoration: none'})
pre_subjhrefs

[<a href="http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&amp;tvd=100100031793509&amp;vrn=100100031793505&amp;region=0&amp;global=1&amp;sub_region=0&amp;prver=0&amp;pronetvd=null&amp;vibid=100100031793851&amp;type=227" style="text-decoration: none">Республика Адыгея (Адыгея)</a>,
 <a href="http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&amp;tvd=100100031793509&amp;vrn=100100031793505&amp;region=0&amp;global=1&amp;sub_region=0&amp;prver=0&amp;pronetvd=null&amp;vibid=100100031793910&amp;type=227" style="text-decoration: none">Республика Алтай</a>,
 <a href="http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&amp;tvd=100100031793509&amp;vrn=100100031793505&amp;region=0&amp;global=1&amp;sub_region=0&amp;prver=0&amp;pronetvd=null&amp;vibid=100100031793852&amp;type=227" style="text-decoration: none">Республика Башкортостан</a>,
 <a href="http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&amp;tvd=100100031793509&amp;vrn=100100031793

Отлично! У нас есть список ссылок. Обработаем каждую из них. Нас интересует ссылка и название субъекта. 

In [7]:
pre_subjhrefs[0]

<a href="http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&amp;tvd=100100031793509&amp;vrn=100100031793505&amp;region=0&amp;global=1&amp;sub_region=0&amp;prver=0&amp;pronetvd=null&amp;vibid=100100031793851&amp;type=227" style="text-decoration: none">Республика Адыгея (Адыгея)</a>

In [8]:
pre_subjhrefs[0].text

'Республика Адыгея (Адыгея)'

In [9]:
# Вопрос: Почему выдаёт ошибку? Мы же вроде бы всё делаем правильно...
re.split('<a href="|" style=',pre_subjhrefs[0])

TypeError: expected string or bytes-like object

In [10]:
re.split('<a href="|" style=',str(pre_subjhrefs[0]))[1]

'http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&amp;tvd=100100031793509&amp;vrn=100100031793505&amp;region=0&amp;global=1&amp;sub_region=0&amp;prver=0&amp;pronetvd=null&amp;vibid=100100031793851&amp;type=227'

`http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&tvd=100100031793509&vrn=100100031793505&`
`region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=100100031793851&type=227`

Рабочая ссылка отличается от нерабочей какими-то стнными "amp", которые вообще непонятно откуда взялись после амперсандов. Надо бы избавиться от них...

In [11]:
re.sub('amp;', '',re.split('<a href="|" style=',str(pre_subjhrefs[0]))[1])

'http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&tvd=100100031793509&vrn=100100031793505&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=100100031793851&type=227'

Отлично! Теперь вроде бы ссылка рабочая. Попробуем достать ссылки для всех оставшихся республик.

In [12]:
subjects = [[item.text,re.sub('amp;', '',re.split('<a href="|" style=',str(item))[1])] for item in pre_subjhrefs]
subjects

[['Республика Адыгея (Адыгея)',
  'http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&tvd=100100031793509&vrn=100100031793505&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=100100031793851&type=227'],
 ['Республика Алтай',
  'http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&tvd=100100031793509&vrn=100100031793505&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=100100031793910&type=227'],
 ['Республика Башкортостан',
  'http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&tvd=100100031793509&vrn=100100031793505&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=100100031793852&type=227'],
 ['Республика Бурятия',
  'http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&tvd=100100031793509&vrn=100100031793505&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=100100031793853&type=227'],
 ['Республика Дагестан',
  'http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&tvd=100100031793509&

__Теперь, когда мы убедились, что всё работает, оформим весь код выше как функцию.__ 

In [13]:
def subject_hrefer(url):
    
    # Три базовых запроса
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html,"lxml")
    
    # Поиск в html нужного куска
    pre_subjhrefs = soup.findAll("a",{'style':'text-decoration: none'})
    
    # Генерация вектора по принципу объект: ссылка.
    subjects = [[item.text,re.sub('amp;', '',re.split('<a href="|" style=',str(item))[1])] for item in pre_subjhrefs]

    # Возвращаем список объектов-ссылок.
    return(subjects) 

subjects = subject_hrefer(start_page_1)

Пришло время узнать всю информацию о более маленьких участках. Переходим по одной из ссылок на субъект, как рьычно через inspect находим на страничке нужное место с ссылкой и видим, что оно устроено один в один как на предыдущем этапе. А что если применить к этой странице ту же самую функцию?! 

In [14]:
aim_page = subjects[0][1]
subject_hrefer(aim_page)

[['Адыгейская',
  'http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&tvd=100100031793851&vrn=100100031793505&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=2012000191040&type=227'],
 ['Гиагинская',
  'http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&tvd=100100031793851&vrn=100100031793505&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=2012000191041&type=227'],
 ['Кошехабльская',
  'http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&tvd=100100031793851&vrn=100100031793505&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=2012000191042&type=227'],
 ['Красногвардейская',
  'http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&tvd=100100031793851&vrn=100100031793505&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=2012000191043&type=227'],
 ['Майкопская районная ',
  'http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&tvd=100100031793851&vrn=100100031793505&region=0&global=1&su

__ПОЛУЧИЛОСЬ! ЭТО СРАБОТАЛО! УРААА!!!__ Пробегаемся по списку subjects той же самой функцией и получаем кучу новых списков...

In [15]:
subject_subject_hrefs = [subject_hrefer(item[1]) for item in subjects]

Отлично! Мы наконец то добрались до данных по избирательным участкам. Обратим внимание на то, что последние два элемента в скачавшемся списке пустые. Они отвечают за два экстраординарных объекта: Байконур и территорию за пределами РФ. Пока что выбросим их из нашего анализа.

На текущих страничках нас будет интересовать одна - единственная ссылка, которая ведёт на сайт избирательной комиссии субъекта. Всего таких сайтов будет больше 80 штук. Помолимся создателю, чтобы их страницы имели сходную структуру.

In [16]:
aim_url = subject_subject_hrefs[0][0][1]
aim_url

'http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&tvd=100100031793851&vrn=100100031793505&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=2012000191040&type=227'

In [17]:
response = requests.get(aim_url)
html = response.content
soup = BeautifulSoup(html,"lxml")

In [18]:
soup.findAll('a')

[<a class="ib-header-cik" href="http://cikrf.ru">
 <img alt="" src="header/00/header_bg.jpg"/>
 </a>,
 <a class="ib-header-link" href="http://www.vybory.izbirkom.ru/region/izbirkom">Выборы, референдумы и иные формы прямого волеизъявления</a>,
 <a href="region/izbirkom?action=show&amp;root_a=null&amp;vrn=100100031793505&amp;region=0&amp;global=1&amp;type=0&amp;prver=0&amp;pronetvd=null">ЦИК России</a>,
 <a href="region/izbirkom?action=show&amp;root_a=null&amp;vrn=100100031793505&amp;region=0&amp;global=1&amp;type=0&amp;root=1000022&amp;prver=0&amp;pronetvd=null&amp;tvd=100100031793851">Республика Адыгея (Адыгея)</a>,
 <a href="region/izbirkom?action=show&amp;root_a=null&amp;vrn=100100031793505&amp;region=0&amp;global=1&amp;type=0&amp;root=12000001&amp;prver=0&amp;pronetvd=null&amp;tvd=2012000191040">Адыгейская</a>,
 <a href="http://www.vybory.izbirkom.ru/region/izbirkom?action=show&amp;global=true&amp;root=12000001&amp;tvd=2012000191040&amp;vrn=100100031793505&amp;prver=0&amp;pronetvd=n

Получаем огромное количество ссылок. Придумывать как выдернуть конкретную как-то лениво. Будем молиться, что все странички будут иметь одинаковую структуру и нужная нам ссылка будет второй снизу... Вытащим её по аналогии с тем, что мы делали ранее.

In [19]:
re.sub('amp;','',re.split('href="|">',str(soup.findAll('a')[-2]))[1])

'http://www.vybory.izbirkom.ru/region/izbirkom?action=show&global=true&root=12000001&tvd=2012000191040&vrn=100100031793505&prver=0&pronetvd=null&region=1&sub_region=1&type=227&vibid=2012000191040'

Оформим всё это дело как функцию для одного урла.

In [20]:
def region_href(url):
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html,"lxml")
    href = re.sub('amp;','',re.split('href="|">',str(soup.findAll('a')[-2]))[1])
    return(href)

Вытащим все такие ссылки. И добавим их в тот же самый вектор.

In [ ]:
for item in subject_subject_hrefs: # цикл по субъектам
    for jtem in item: # цикл по районам субъекта
        aim_url = jtem[1]
        href = region_href(aim_url)
        jtem.append(href)   

Теперь в нашем списке каждому району соответствует две ссылки. Одна ведёт на страницу со сводной таблицой по этому району, а вторая на страницу с детальной информацией по УИКам, которую мы так жаждем получить. Вся информация, доступная на первой ссылке дублируется по второй. Не очень понятно зачем вообще она нужна...

In [23]:
subject_subject_hrefs[0][0]

['Адыгейская',
 'http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&tvd=100100031793851&vrn=100100031793505&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=2012000191040&type=227',
 'http://www.vybory.izbirkom.ru/region/izbirkom?action=show&global=true&root=12000001&tvd=2012000191040&vrn=100100031793505&prver=0&pronetvd=null&region=1&sub_region=1&type=227&vibid=2012000191040']

Вытаскиваем по каждой ссылке табличку с результатами голосований на участках.

In [25]:
url = subject_subject_hrefs[0][0][2]

response = requests.get(url)
html = response.content
soup = BeautifulSoup(html,"lxml")

In [33]:
tablecontent = soup.findAll('table',{'style':"width:100%;border-color:#000000"})
tablecontent = tablecontent[0]  # Помним о том, что findAll на выход наст вектор. В данном случае из 1 элемента. 

In [70]:
# Две таблички. Нас интересует кусочек с УИКами. 
UIK = tablecontent.findAll('table')[-1]

Немного посмотрим на разметку и заметим, что каждая строчка выделяется тэгом tr, а каждый элемнт в ней тэгом td. Вытащим каждую строчку, а после каждый элемент из каждой строчки.

In [75]:
# вытаскиваем все строки
rows = UIK.find_all('tr')
rows[0] 

<tr bgcolor="#FFFFFF" valign="top">
<td align="center" style="color:black">
<nobr>УИК №1</nobr>
</td>
<td align="center" style="color:black">
<nobr>УИК №2</nobr>
</td>
<td align="center" style="color:black">
<nobr>УИК №3</nobr>
</td>
<td align="center" style="color:black">
<nobr>УИК №4</nobr>
</td>
<td align="center" style="color:black">
<nobr>УИК №5</nobr>
</td>
<td align="center" style="color:black">
<nobr>УИК №6</nobr>
</td>
</tr>

In [76]:
rows[0].find_all('td')

[<td align="center" style="color:black">
 <nobr>УИК №1</nobr>
 </td>, <td align="center" style="color:black">
 <nobr>УИК №2</nobr>
 </td>, <td align="center" style="color:black">
 <nobr>УИК №3</nobr>
 </td>, <td align="center" style="color:black">
 <nobr>УИК №4</nobr>
 </td>, <td align="center" style="color:black">
 <nobr>УИК №5</nobr>
 </td>, <td align="center" style="color:black">
 <nobr>УИК №6</nobr>
 </td>]

In [82]:
rows[0].find_all('td')[0]

<td align="center" style="color:black">
<nobr>УИК №1</nobr>
</td>

In [83]:
rows[0].find_all('td')[0].text

'\nУИК №1\n'

In [84]:
rows[0].find_all('td')[0].text.strip()

'УИК №1'

Такимии незатейливыми действиями вытащили отдельный объект из отдельной строки. А теперь всё сразу в виде двух циклов:

In [97]:
rows = UIK.find_all('tr')
data = [ ]
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele])
data = pd.DataFrame(data)
data.head()

,0,1,2,3,4,5
0,УИК №1,УИК №2,УИК №3,УИК №4,УИК №5,УИК №6
1,2383,2865,2821,2069,777,1106
2,2147,2586,2558,1868,705,965
3,0,0,0,0,0,0
4,1513,1764,1604,1176,435,795


Теперь по аналогии нужно будет выковыривать первый кусочек таблицы. Поэтому оформим всё, что было написано выше в виде функции. Это удобно, к этому надо привыкнуть, если вы ещё не привыкли! 

In [259]:
def TableCreator(html_table):
    rows = html_table.find_all('tr')
    data = [ ]
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele])
    return(data)

In [260]:
Table1 = TableCreator(tablecontent.findAll('table')[-1])
Table2 = TableCreator(tablecontent.findAll('table')[0])

In [261]:
pd.DataFrame(Table2)

,0,1,2
0,Сумма,None,None
1,1,"Число избирателей, включенных в список избират...",12021
2,2,"Число избирательных бюллетеней, полученных уча...",10829
3,3,"Число избирательных бюллетеней, выданных избир...",0
4,4,"Число избирательных бюллетеней, выданных в пом...",7287
5,5,"Число избирательных бюллетеней, выданных вне п...",497
6,6,Число погашенных избирательных бюллетеней,3045
7,7,Число избирательных бюллетеней в переносных ящ...,497
8,8,Число бюллетеней в стационарных ящиках для гол...,7279
9,9,Число недействительных избирательных бюллетеней,95


In [232]:
Table2[0].insert(0,'Номер участка') 
Table2[0].insert(0,'Какая-то цифра')

Переходим в Pandas. Сделаем из первой строки для каждой таблицы заголовок. Из второй таблицы удалим лишний столбец. После объединим все таблицы в единое целое.

In [235]:
Tbl1 = pd.DataFrame(Table1[1:],columns=Table1[0])
Tbl2 = pd.DataFrame(Table2[1:],columns=Table2[0])
Tbl2 = Tbl2.drop('Какая-то цифра',axis=1)

print(Tbl1.shape)
print(Tbl2.shape)  # По строкам размер таблиц совпадает

itog_table = Tbl2.join(Tbl1) # Объединяем таблички
itog_table

(24, 6)
(24, 2)


,Номер участка,Сумма,УИК №1,УИК №2,УИК №3,УИК №4,УИК №5,УИК №6
0,"Число избирателей, включенных в список избират...",12021,2383,2865,2821,2069,777,1106
1,"Число избирательных бюллетеней, полученных уча...",10829,2147,2586,2558,1868,705,965
2,"Число избирательных бюллетеней, выданных избир...",0,0,0,0,0,0,0
3,"Число избирательных бюллетеней, выданных в пом...",7287,1513,1764,1604,1176,435,795
4,"Число избирательных бюллетеней, выданных вне п...",497,77,98,106,70,27,119
5,Число погашенных избирательных бюллетеней,3045,557,724,848,622,243,51
6,Число избирательных бюллетеней в переносных ящ...,497,77,98,106,70,27,119
7,Число бюллетеней в стационарных ящиках для гол...,7279,1513,1762,1598,1176,435,795
8,Число недействительных избирательных бюллетеней,95,19,29,31,0,8,8
9,Число действительных избирательных бюллетеней,7681,1571,1831,1673,1246,454,906


Видим, что нужно выбросить 18 строку и отредактировать абсолютные и относительные доли за кандидатов. В таблице на сайте они неудачно оформлены в две строки. 

Начинаем пределывать таблицу в нужный нам околопанельный формат. Попутно исправляем все баги. Это будет весело (нет).

In [242]:
itog_table = Tbl2.join(Tbl1)     # Объединяем таблички

itog_table = itog_table.drop(18) # Выкидываем 18 строку

# Изготовим с помощью костылей названия колонок и перевернём табличку
col_names = itog_table['Номер участка']
itog_table = itog_table.drop('Номер участка',axis=1)
itog_table = itog_table.drop('Сумма',axis=1)
itog_table = itog_table.T
# Вернули имена
itog_table.columns = col_names

# Добавляем дополнительный столбец с названием участка

itog_table[]
subject_subject_hrefs[0][0][0]

'Адыгейская'

Осталось пофиксить последние колонки.

In [245]:
itog_table

Номер участка,"Число избирателей, включенных в список избирателей","Число избирательных бюллетеней, полученных участковой избирательной комиссией","Число избирательных бюллетеней, выданных избирателям, проголосовавшим досрочно","Число избирательных бюллетеней, выданных в помещении для голосования в день голосования","Число избирательных бюллетеней, выданных вне помещения для голосования в день голосования",Число погашенных избирательных бюллетеней,Число избирательных бюллетеней в переносных ящиках для голосования,Число бюллетеней в стационарных ящиках для голосования,Число недействительных избирательных бюллетеней,Число действительных избирательных бюллетеней,...,Число неиспользованных открепительных удостоверений,"Число открепительных удостоверений, выданных избирателям ТИК",Число утраченных открепительных удостоверений,Число утраченных избирательных бюллетеней,"Число избирательных бюллетеней, не учтенных при получении",Жириновский Владимир Вольфович,Зюганов Геннадий Андреевич,Миронов Сергей Михайлович,Прохоров Михаил Дмитриевич,Путин Владимир Владимирович
УИК №1,2383,2147,0,1513,77,557,77,1513,19,1571,...,11,1,0,0,0,24\n1.51%,382\n24.03%,28\n1.76%,71\n4.47%,1066\n67.04%
УИК №2,2865,2586,0,1764,98,724,98,1762,29,1831,...,0,1,0,0,0,51\n2.74%,453\n24.35%,49\n2.63%,104\n5.59%,1174\n63.12%
УИК №3,2821,2558,0,1604,106,848,106,1598,31,1673,...,6,4,0,0,0,36\n2.11%,481\n28.23%,24\n1.41%,107\n6.28%,1025\n60.15%
УИК №4,2069,1868,0,1176,70,622,70,1176,0,1246,...,11,1,0,0,0,0\n0.00%,414\n33.23%,0\n0.00%,48\n3.85%,784\n62.92%
УИК №5,777,705,0,435,27,243,27,435,8,454,...,1,0,0,0,0,19\n4.11%,138\n29.87%,4\n0.87%,7\n1.52%,286\n61.90%
УИК №6,1106,965,0,795,119,51,119,795,8,906,...,16,3,0,0,0,7\n0.77%,104\n11.38%,10\n1.09%,13\n1.42%,772\n84.46%


Всё! Есть две таблицы. Мы приходим по ссылке, забираем их, соединяем в одно целое и уходим. После объединяем всё это в красивенький датафрейм и всё! Данные наши. Добьём всё это дело в одну функцию. 

In [ ]:
def Super_puper_table_uniter_and_creator(url):
    # Забираем всё с сайта региональной комиссии
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html,"lxml")
    
    # Находим таблицу
    tablecontent = soup.findAll('table',{'style':"width:100%;border-color:#000000"})[0]
    # Помним о том, что findAll на выход наст вектор. В данном случае из 1 элемента. 
    
    # Вытаскиваем две таблицы: 
    Table1 = TableCreator(tablecontent.findAll('table')[-1])
    Table2 = TableCreator(tablecontent.findAll('table')[0])
    
    # Создаём одну красивенькую.
    

In [98]:
UIK2 = tablecontent.findAll('table')[0]

Наш итоговый код получился совсем небольшим и довольно красивым. Можно собирать результаты выборов в большие таблички и приступать к их анализу. 

In [ ]:
import re
import pandas as pd

start_page_1 = "http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&root=1&tvd=100100031793509&vrn=100100031793505&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=100100031793509&type=227"

# Функция для первых двух шагов по сайту ЦИК
def subject_hrefer(url):
    
    # Три базовых запроса
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html,"lxml")
    
    # Поиск в html нужного куска
    pre_subjhrefs = soup.findAll("a",{'style':'text-decoration: none'})
    
    # Генерация вектора по принципу объект: ссылка
    subjects = [[item.text,re.sub('amp;', '',re.split('<a href="|" style=',str(item))[1])] for item in pre_subjhrefs]

    # Возвращаем список объектов-ссылок
    return(subjects) 

# Собираем все субъекты
subjects = subject_hrefer(start_page_1)

# Собираем все участки 
subject_subject_hrefs = [subject_hrefer(item[1]) for item in subjects]

# Собираем все ссылки на таблички с УИКами по районам
def region_href(url):
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html,"lxml")
    href = re.sub('amp;','',re.split('href="|">',str(soup.findAll('a')[-2]))[1])
    return(href)

for item in subject_subject_hrefs: # цикл по субъектам
    for jtem in item: # цикл по районам субъекта
        aim_url = jtem[1]
        href = region_href(aim_url)
        jtem.append(href)  

# Поиск и вытаскивание таблиц по разметке html:
def TableCreator(html_table):
    rows = html_table.find_all('tr')
    data = [ ]
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele])
    return(data)





Зависимость результатов выборов от явки. Ось абцисс - явка в процентах, ось ординат - процент голосов от общего зарегистрированного числа голосов на участке с таким показателем явки.

Объяснение 1: электорат оппозиции приходит весь, электорат проправительственной партии более пассивен. Поэтому их доля в явке может варьироваться в больших диапазонах.

Объяснение 2: вбросы, административное давление (рота солдат пришла голосовать, давление на врачей и других бюджетников) 

Гистограмма по России. Пики в красивых точках. 